# Demo of running a Flyte + Feast, feature engineering and training pipeline
In this demo we will learn how to interact with Feast through Flyte. The goal will be to train a simple [Gaussian Naive Bayes model using sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html) on the [Horse-Colic dataset from UCI](https://archive.ics.uci.edu/ml/datasets/Horse+Colic).

**NOTE**
We will not really dive into the dataset or the model, as the aim of this tutorial is to show how you can use Feast as the feature store and use Flyte to engineer the features that can be identical across your online predictions as well as offline training

## Step 1: Check out the code for the pipeline
We have used [flytekit](https://docs.flyte.org/projects/flytekit/en/latest/) flyte's python SDK to express the pipeline in pure python. The code is auto-documented and rendered using sphinx [here]()

## Step 2: Launch an execution
We can use the [FlyteConsole](https://github.com/flyteorg/flyteconsole) to launch, monitor and introspect Flyte executions, but in this case we will use [flytekit.remote](https://docs.flyte.org/projects/flytekit/en/latest/design/control_plane.html) to interact with the Flyte backend.

In [3]:
from flytekit.remote import FlyteRemote
remote = FlyteRemote.from_config("flytesnacks", "development")

Using default config file at /Users/ketanumare/.flyte/config


{"asctime": "2021-09-28 09:44:07,041", "name": "flytekit.remote", "levelname": "WARNING", "message": "This feature is still in beta. Its interface and UX is subject to change."}


No images specified, will use the default image


In [49]:
# from feast_integration.feast_workflow import feast_workflow
lp = remote.fetch_launch_plan(name="feast_integration.feast_workflow.feast_workflow")
lp.id.version

'77cf08a7fff024aefd22e927eba6ab7be8323293'

In [50]:
from feast_workflow import feast_workflow
lp.guessed_python_interface = feast_workflow.python_interface
exe = remote.execute(lp, inputs={})

In [51]:
print(f"http://localhost:30081/console/projects/{exe.id.project}/domains/{exe.id.domain}/executions/{exe.id.name}")

http://localhost:30081/console/projects/flytesnacks/domains/development/executions/fdde7d53867b74cd9885


In [52]:
exe = remote.sync(exe)

In [53]:
exe.sync()

In [54]:
feature_store = exe.outputs['o0']
model = exe.outputs['o1']
predictions = exe.outputs['o2']

In [4]:
exe2 = remote.fetch_workflow_execution(name='fdde7d53867b74cd9885')
exe2 = remote.sync(exe2)
from feast_dataobjects import FeatureStore
fs = exe2.raw_outputs.get('o0', FeatureStore)

In [5]:
from feast_workflow import predict, FEAST_FEATURES

/Users/ketanumare/.virtualenvs/flyte-feast/lib/python3.8/site-packages/marshmallow/fields.py:173: RemovedInMarshmallow4Warning: The 'default' argument to fields is deprecated. Use 'dump_default' instead.
  warnings.warn(
/Users/ketanumare/.virtualenvs/flyte-feast/lib/python3.8/site-packages/marshmallow/fields.py:218: RemovedInMarshmallow4Warning: Passing field metadata as a keyword arg is deprecated. Use the explicit `metadata=...` argument instead.
  warnings.warn(
/Users/ketanumare/.virtualenvs/flyte-feast/lib/python3.8/site-packages/marshmallow/fields.py:438: RemovedInMarshmallow4Warning: The 'default' attribute of fields is deprecated. Use 'dump_default' instead.
  warnings.warn(


In [6]:
fs1 = fs._build_feast_feature_store()
fs1

In [ ]:

inference_point = fs.get_online_features(FEAST_FEATURES, [{"Hospital Number": 5290409}])


ERROR:root:Error from command '['aws', 's3', 'cp', 's3://feast-integration/online.db', 'online.db']':
b'fatal error: An error occurred (403) when calling the HeadObject operation: Forbidden\n'

ERROR:root:Exception when trying to execute ['aws', 's3', 'cp', 's3://feast-integration/online.db', 'online.db'], reason: Called process exited with error code: 1.  Stderr dump:

b'fatal error: An error occurred (403) when calling the HeadObject operation: Forbidden\n'


In [40]:
predict(model, inference_point)

In [2]:
from flytekit.configuration import aws
from flytekit.configuration import set_flyte_config_file
set_flyte_config_file('/Users/ketanumare/.flyte/config')
print(aws.S3_ACCESS_KEY_ID.get())
print(aws.S3_ENDPOINT.get())
print(aws.S3_SECRET_ACCESS_KEY.get())

minio
http://localhost:30084
miniostorage


In [1]:
!cat /Users/ketanumare/.flyte/config

[platform]
url=localhost:30081
insecure=True

[credentials]
client_id=flytectl
redirect_uri=http://localhost:53593/callback
oauth_scopes=offline,all
authorization_metadata_key=flyte-authorization
auth_mode=standard

[aws]
s3_endpoint=http://localhost:30084
s3_access_key_id=minio
s3_secret_access_key=miniostorage
